In [ ]:
!pip install --upgrade "ultralytics==8.3.165"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path to dataset zip in Drive
ZIP_PATH = '/content/drive/MyDrive/Data_Frames.zip'

!unzip -q "$ZIP_PATH" -d /content
DATASET_ROOT = '/content/Data_Frames'

In [ ]:
#  Batch fine-tune with RandomShadow AUG (YAML-embedded)
#  — Colby Bowie, July 2025
#
#  * Trains the seven Fixed_Num models
#  * Evaluates each on day / shop / led test sets
#  * Outputs everything to /Synthetic_shadow_fixed_results

from ultralytics import YOLO
from pathlib import Path
import yaml, shutil, gc

# ---------- PATHS ----------------------------------------------------------
BASELINE_ROOT = Path("/content/drive/MyDrive/Baseline_fixed_results")
SHADOW_ROOT   = Path("/content/drive/MyDrive/Synthetic_shadow_fixed_results")
SHADOW_ROOT.mkdir(parents=True, exist_ok=True)

FIXED_CFGS = [
    "day_only", "shop_only", "led_only",
    "day_shop", "day_led", "shop_led", "all_lighting",
]

# ---------- Albumentations config to inject --------------------------------------
SHADOW_AUG = [{
    "RandomShadow": {
        "num_shadows_lower": 1,
        "num_shadows_upper": 1,
        "shadow_dimension": 0.5,
        "shadow_roi": [0, 0, 1, 1],
        "p": 0.30
    }
}]

# ---------- Helper: build training YAML with aug ---------------------------------
def make_train_yaml(src_yaml: Path, dst_yaml: Path):
    data = yaml.safe_load(src_yaml.read_text())
    data["albumentations"] = SHADOW_AUG
    dst_yaml.parent.mkdir(parents=True, exist_ok=True)
    yaml.safe_dump(data, dst_yaml.open("w"))
    return dst_yaml

# ---------- Loop over all baselines ----------------------------------------------
for run in FIXED_CFGS:
    base_dir   = BASELINE_ROOT / run
    best_ckpt  = base_dir / "weights/best.pt"
    base_yaml  = base_dir / f"data_{run}.yaml"
    assert best_ckpt.exists() and base_yaml.exists(), f"Missing files for {run}"

    out_dir    = SHADOW_ROOT / run
    train_yaml = make_train_yaml(base_yaml, out_dir / f"data_{run}_shadow.yaml")

    print(f"\n Fine-tuning {run} with RandomShadow augmentation")

    model = YOLO(str(best_ckpt))
    model.train(
        data=str(train_yaml),
        epochs=100,
        patience=20,
        imgsz=640,
        batch=-1,
        device=0,
        project=str(SHADOW_ROOT),
        name=run,
        exist_ok=True,
        resume=False,
        save_period=20,
    )

    # ------------- evaluate on day / shop / led (no aug) -------------------
    TEST_SPLITS = {
        "day_test":  "Testing/day_test/images",
        "shop_test": "Testing/shop_test/images",
        "led_test":  "Testing/led_test/images",
    }
    train_dir = Path("/content/Data_Frames/Fixed_Num") / run / "train/images"
    val_dir   = Path("/content/Data_Frames/Fixed_Num") / run / "val/images"

    def simple_yaml(test_rel, dst):
        data = {
            "train": str(train_dir),
            "val":   str(val_dir),
            "test":  str(Path("/content/Data_Frames") / test_rel),
            "nc": 1,
            "names": ["spaghetti"]
        }
        yaml.safe_dump(data, Path(dst).open("w"))
        return dst

    best_shadow = out_dir / "weights/best.pt"
    eval_model  = YOLO(str(best_shadow))

    for tag, rel in TEST_SPLITS.items():
        yml = simple_yaml(rel, out_dir / f"eval_{run}_{tag}.yaml")
        m   = eval_model.val(
                data=str(yml), split="test",
                imgsz=640, device=0,
                save_json=True,
                project=str(out_dir),
                name=f"eval_{run}_{tag}",
                verbose=False)
        print(f" {run:10s}  on {tag:9s}  mAP50={m.box.map50:.3%}  mAP95={m.box.map:.3%}")

    del model, eval_model; gc.collect()

print("\n All shadow-aug baselines complete.  Results in:", SHADOW_ROOT)

In [ ]:
#  re-evaluate all models
#  — Colby Bowie, July 2025
#
#  • Finds each best.pt in Synthetic_shadow_fixed_results
#  • Re-runs YOLOv9 evaluation on every YAML that has a test split
#  • Saves new results under reval_<run>_<tag>/
#  • Builds a summary CSV (model × test_split × mAP50/95)

from pathlib import Path
import yaml
import pandas as pd
from ultralytics import YOLO
import gc, re

# ---------- CONFIG ---------------------------------------------------------
RUNS_ROOT   = Path("/content/drive/MyDrive/Synthetic_shadow_fixed_results")   # << change if needed
DEVICE      = 0     # GPU id, -1 for CPU
IMG_SIZE    = 640   # use same resolution as training
SAVE_JSON   = True  # save COCO json for each eval

# ----------- RUN -----------------------------------------------------------
rows = []

for run_dir in sorted([p for p in RUNS_ROOT.iterdir() if p.is_dir()]):
    best_pt = run_dir / "weights" / "best.pt"
    if not best_pt.exists():
        print(f" No best.pt in {run_dir.name} — skipping")
        continue

    # grab all YAML files in run folder (direct children or eval_* sub-dirs)
    yaml_paths = list(run_dir.glob("*.yaml")) + list(run_dir.glob("eval_*/*.yaml"))
    if not yaml_paths:
        print(f" No YAMLs found in {run_dir.name}")
        continue

    model = YOLO(str(best_pt))
    print(f"\n Evaluating {run_dir.name}  ({len(yaml_paths)} YAMLs found)")

    for yml in sorted(yaml_paths):
        with open(yml, "r") as f:
            data_cfg = yaml.safe_load(f)

        # skip if YAML lacks a test path
        if "test" not in data_cfg or not data_cfg["test"]:
            continue

        # derive a tag from YAML filename, e.g. eval_day_only_shop_test.yaml ➜ shop_test
        tag_match = re.search(r"(day_test|shop_test|led_test)", yml.stem)
        tag = tag_match.group(1) if tag_match else yml.stem

        out_name = f"reval_{run_dir.name}_{tag}"
        if (run_dir / out_name / "results.csv").exists():
            print(f"↻  {out_name} already exists — skipping reevaluation")
            continue

        metrics = model.val(
            data=str(yml),
            split="test",
            imgsz=IMG_SIZE,
            device=DEVICE,
            save_json=SAVE_JSON,
            project=str(run_dir),
            name=out_name,
            verbose=False,
        )

        # collect for summary
        rows.append({
            "model": run_dir.name,
            "test_split": tag,
            "mAP50":  metrics.box.map50 * 100,
            "mAP95":  metrics.box.map * 100,
            "precision": metrics.box.mp * 100,   # mean precision  (mp)
            "recall":    metrics.box.mr * 100    # mean recall     (mr)
        })

    del model; gc.collect()

# ---------- build summary table -------------------------------------------
if rows:
    df = pd.DataFrame(rows).sort_values(["model", "test_split"])
    df.to_csv(RUNS_ROOT / "cross_light_summary.csv", index=False)
    print("\n Summary saved to cross_light_summary.csv\n")
    print(df.to_string(index=False, formatters={"mAP50": "{:.1f}".format,
                                                "mAP95": "{:.1f}".format,
                                                "precision": "{:.1f}".format,
                                                "recall": "{:.1f}".format}))
else:
    print(" No evaluations were run — check YAML/test paths.")

In [ ]:
#  Batch fine-tune with RandomShadow AUG (YAML-embedded)
#  — Colby Bowie, July 2025
#
#  * Trains the seven Best_Num models
#  * Evaluates each on day / shop / led test sets
#  * Outputs everything to /Synthetic_shadow_best_results

from ultralytics import YOLO
from pathlib import Path
import yaml, shutil, gc

# ---------- Paths -----------------------------------------------------------------
BASELINE_ROOT = Path("/content/drive/MyDrive/Baseline_best_results")
SHADOW_ROOT   = Path("/content/drive/MyDrive/Synthetic_shadow_best_results")
SHADOW_ROOT.mkdir(parents=True, exist_ok=True)

FIXED_CFGS = [
    "day_shop", "day_led", "shop_led", "all_lighting",
]

# ---------- Albumentations config to inject --------------------------------------
SHADOW_AUG = [{
    "RandomShadow": {
        "num_shadows_lower": 1,
        "num_shadows_upper": 1,
        "shadow_dimension": 0.5,
        "shadow_roi": [0, 0, 1, 1],
        "p": 0.30
    }
}]

# ---------- Helper: build training YAML with aug ---------------------------------
def make_train_yaml(src_yaml: Path, dst_yaml: Path):
    data = yaml.safe_load(src_yaml.read_text())
    data["albumentations"] = SHADOW_AUG
    dst_yaml.parent.mkdir(parents=True, exist_ok=True)
    yaml.safe_dump(data, dst_yaml.open("w"))
    return dst_yaml

# ---------- Loop over all baselines ----------------------------------------------
for run in FIXED_CFGS:
    base_dir   = BASELINE_ROOT / run
    best_ckpt  = base_dir / "weights/best.pt"
    base_yaml  = base_dir / f"data_{run}.yaml"
    assert best_ckpt.exists() and base_yaml.exists(), f"Missing files for {run}"

    out_dir    = SHADOW_ROOT / run
    train_yaml = make_train_yaml(base_yaml, out_dir / f"data_{run}_shadow.yaml")

    print(f"\n Fine-tuning {run} with RandomShadow augmentation")

    model = YOLO(str(best_ckpt))
    model.train(
        data=str(train_yaml),
        epochs=100,
        patience=20,
        imgsz=640,
        batch=-1,
        device=0,
        project=str(SHADOW_ROOT),
        name=run,
        exist_ok=True,
        resume=False,
        save_period=20,
    )

    # ------------- evaluate on day / shop / led (no aug) -------------------
    TEST_SPLITS = {
        "day_test":  "Testing/day_test/images",
        "shop_test": "Testing/shop_test/images",
        "led_test":  "Testing/led_test/images",
    }
    train_dir = Path("/content/Data_Frames/Best_case") / run / "train/images"
    val_dir   = Path("/content/Data_Frames/Best_case") / run / "val/images"

    def simple_yaml(test_rel, dst):
        data = {
            "train": str(train_dir),
            "val":   str(val_dir),
            "test":  str(Path("/content/Data_Frames") / test_rel),
            "nc": 1,
            "names": ["spaghetti"]
        }
        yaml.safe_dump(data, Path(dst).open("w"))
        return dst

    best_shadow = out_dir / "weights/best.pt"
    eval_model  = YOLO(str(best_shadow))

    for tag, rel in TEST_SPLITS.items():
        yml = simple_yaml(rel, out_dir / f"eval_{run}_{tag}.yaml")
        m   = eval_model.val(
                data=str(yml), split="test",
                imgsz=640, device=0,
                save_json=True,
                project=str(out_dir),
                name=f"eval_{run}_{tag}",
                verbose=False)
        print(f" {run:10s}  on {tag:9s}  mAP50={m.box.map50:.3%}  mAP95={m.box.map:.3%}")

    del model, eval_model; gc.collect()

print("\n All shadow-aug baselines complete.  Results in:", SHADOW_ROOT)

In [ ]:
#  re-evaluate all models
#  — Colby Bowie, July 2025
#
#  • Finds each best.pt in Synthetic_shadow_best_results
#  • Re-runs YOLOv9 evaluation on every YAML that has a test split
#  • Saves new results under reval_<run>_<tag>/
#  • Builds a summary CSV (model × test_split × mAP50/95)


from pathlib import Path
import yaml
import pandas as pd
from ultralytics import YOLO
import gc, re

# ---------- CONFIG ---------------------------------------------------------
RUNS_ROOT   = Path("/content/drive/MyDrive/Synthetic_shadow_best_results")   # << change if needed
DEVICE      = 0     # GPU id, -1 for CPU
IMG_SIZE    = 640   # use same resolution as training
SAVE_JSON   = True  # save COCO json for each eval

# ----------- RUN -----------------------------------------------------------
rows = []

for run_dir in sorted([p for p in RUNS_ROOT.iterdir() if p.is_dir()]):
    best_pt = run_dir / "weights" / "best.pt"
    if not best_pt.exists():
        print(f" No best.pt in {run_dir.name} — skipping")
        continue

    # grab all YAML files in run folder (direct children or eval_* sub-dirs)
    yaml_paths = list(run_dir.glob("*.yaml")) + list(run_dir.glob("eval_*/*.yaml"))
    if not yaml_paths:
        print(f" No YAMLs found in {run_dir.name}")
        continue

    model = YOLO(str(best_pt))
    print(f"\n Evaluating {run_dir.name}  ({len(yaml_paths)} YAMLs found)")

    for yml in sorted(yaml_paths):
        with open(yml, "r") as f:
            data_cfg = yaml.safe_load(f)

        # skip if YAML lacks a test path
        if "test" not in data_cfg or not data_cfg["test"]:
            continue

        # derive a tag from YAML filename, e.g. eval_day_only_shop_test.yaml ➜ shop_test
        tag_match = re.search(r"(day_test|shop_test|led_test)", yml.stem)
        tag = tag_match.group(1) if tag_match else yml.stem

        out_name = f"reval_{run_dir.name}_{tag}"
        if (run_dir / out_name / "results.csv").exists():
            print(f"↻  {out_name} already exists — skipping reevaluation")
            continue

        metrics = model.val(
            data=str(yml),
            split="test",
            imgsz=IMG_SIZE,
            device=DEVICE,
            save_json=SAVE_JSON,
            project=str(run_dir),
            name=out_name,
            verbose=False,
        )

        # collect for summary
        rows.append({
            "model": run_dir.name,
            "test_split": tag,
            "mAP50":  metrics.box.map50 * 100,
            "mAP95":  metrics.box.map * 100,
            "precision": metrics.box.mp * 100,   # mean precision  (mp)
            "recall":    metrics.box.mr * 100    # mean recall     (mr)
        })

    del model; gc.collect()

# ---------- build summary table -------------------------------------------
if rows:
    df = pd.DataFrame(rows).sort_values(["model", "test_split"])
    df.to_csv(RUNS_ROOT / "cross_light_summary.csv", index=False)
    print("\n Summary saved to cross_light_summary.csv\n")
    print(df.to_string(index=False, formatters={"mAP50": "{:.1f}".format,
                                                "mAP95": "{:.1f}".format,
                                                "precision": "{:.1f}".format,
                                                "recall": "{:.1f}".format}))
else:
    print(" No evaluations were run — check YAML/test paths.")